In [10]:
import requests
from bs4 import BeautifulSoup as bs
import urllib
import gzip
import zlib
import time
import numpy as np
from contextlib import closing
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import copy
import time

In [2]:
MIN_LIKES = 10
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)

shown_posts = 0
collected_posts = 0

In [3]:
def is_like(text):
    return 'ดี' in text

def masking(data, selectors):
    return (d for d, s in zip(data, selectors) if s)

In [6]:
def get_comments(comments):
    if comments is not None:
        for comment in comments:
            try:
                likes = int(comment.find('div', {'class':'display-post-vote'}).text)
                body = comment.find('div', {'class':'display-post-story'}).get_text().strip()
                if likes>=MIN_LIKES and len(body)>0:
                    global collected_post
                    collected_post += 1
                    print(comment.find('span', {'class':'display-post-number'}).text.strip())
                    date,time = comment.find('div', {'class':'display-post-avatar-inner'}).abbr['data-utime'].split()
                    print('Date:', date)
                    print('Time:', time)
                    print('Owner:',comment.find('a', {'class':'display-post-name'}).text)
                    print('Likes:',likes)
                    print('Emotes:',comment.find('div', {'class':'display-post-emotion'}).text.strip())
                    print('Comment:',body.split('แก้ไขข้อความเมื่อ')[0].replace('[Spoil] คลิกเพื่อดูข้อความที่ซ่อนไว้','').strip())
                    print()
            except:
                pass
    return

def get_post_description(post_links):
    for link in post_links:
        url = 'https://pantip.com'+link
        print('URL:',url.strip())
        browser.get(url)
        
        soup = bs(browser.page_source, 'lxml')
        browser.execute_script("window.stop();")
        
        print('Title:',soup.find('h2', {'class':'display-post-title'}).string)
        date,time = soup.find('span', {'class':'display-post-timestamp'}).abbr['data-utime'].split()
        print('Date:', date)
        print('Time:', time)
        print('Tags:',', '.join(soup.find('div', {'class':'display-post-tag-wrapper'}).text.strip().split()))
        print('Post type:',soup.find('div',{'class':'display-post-favourite remark-four-txt'}).text.strip())
        print('Owner:',soup.find('a', {'class':'display-post-name owner'}).text)
        print('Likes:',int(soup.find('div', {'class':'display-post-vote'}).text))
        print('Emotes:',int(soup.find('div', {'class':'display-post-emotion'}).text))
        story = soup.find('div', {'class':'display-post-story'}).text.split('$(document)')[0].strip()
        story = story.split('แก้ไขข้อความเมื่อ')[0].strip()
        print('Story:',story.replace('[Spoil] คลิกเพื่อดูข้อความที่ซ่อนไว้','').strip())
        print()
        comments = soup.find_all('div', {'class':'display-post-wrapper-inner'})
        get_comments(comments)
    

In [ ]:
# req = requests.get('https://pantip.com/forum?tid=30596424')
# req.encoding = 'utf-8'
# soup = bs(req.text, 'lxml')

# all_posts = soup.find_all('div', {'class':'post-item'})

# next_page_url = soup.find('div', {'class':'loadmore-bar indexlist'}).a['href']
# print(next_page_url.split('?')[1])

# # Find post that has more than minimum likes
# liked_posts = [post.find('div', {'class':'post-item-status-i','title':is_like}) for post in all_posts]
# mask = np.array([int(post.get_text()) if post is not None else 0 for post in liked_posts]) >= MIN_LIKES
# liked_posts = list(masking(all_posts,mask))

# links = [post.find('a',href=True)['href'] for post in liked_posts]
# links

# get_post_description(links)

In [11]:
t = time.time()
START_URL = 'https://pantip.com/forum?'
next_page_url = ''
shown_posts = 0
collected_posts = 0
for _ in range(100):
    req = requests.get(START_URL + next_page_url)
    req.encoding = 'utf-8'
    soup = bs(req.text, 'lxml')

    all_posts = soup.find_all('div', {'class':'post-item'})
    shown_posts += len(all_posts)

    next_page_url = soup.find('div', {'class':'loadmore-bar indexlist'}).a['href'].split('?')[1]

    # Find post that has more than minimum likes
    liked_posts = [post.find('div', {'class':'post-item-status-i','title':is_like}) for post in all_posts]
    mask = np.array([int(post.get_text()) if post is not None else 0 for post in liked_posts]) >= MIN_LIKES
    liked_posts = list(masking(all_posts,mask))

    links = [post.find('a',href=True)['href'] for post in liked_posts]
    collected_posts += len(links)

    get_post_description(links)
    print()
    print('Total shown posts:',shown_posts)
    print('Total collected posts:',collected_posts)
    print('Next page:',next_page_url.strip())
print('Ran for:', time.time()-t, 'seconds')


Total shown posts: 50
Total collected posts: 0
Next page: tid=39008902

Total shown posts: 100
Total collected posts: 0
Next page: tid=39008851

Total shown posts: 150
Total collected posts: 0
Next page: tid=39008794

Total shown posts: 200
Total collected posts: 0
Next page: tid=39008740

Total shown posts: 250
Total collected posts: 0
Next page: tid=39008688

Total shown posts: 300
Total collected posts: 0
Next page: tid=39008638

Total shown posts: 350
Total collected posts: 0
Next page: tid=39008587

Total shown posts: 400
Total collected posts: 0
Next page: tid=39008533

Total shown posts: 450
Total collected posts: 0
Next page: tid=39008482
URL: https://pantip.com/topic/39008441
Title: [KPOP] อัลบั้ม 'Map of the Soul: Persona' BTS ได้รับรองยอดขายระดับ Silver ใน UK
Date: 06/28/2019
Time: 21:22:59
Tags: K-POP, BTS, (นักร้องนักดนตรี)
Post type: กระทู้สนทนา
Owner: สมาชิกหมายเลข 1720853
Likes: 19
Emotes: 46
Story: อัลบั้ม 'Map of the Soul: Persona' ได้ certified ระดับ Silver (60,000 


Total shown posts: 800
Total collected posts: 2
Next page: tid=39008117

Total shown posts: 850
Total collected posts: 2
Next page: tid=39008065

Total shown posts: 900
Total collected posts: 2
Next page: tid=39008014

Total shown posts: 950
Total collected posts: 2
Next page: tid=39007963

Total shown posts: 1000
Total collected posts: 2
Next page: tid=39007912

Total shown posts: 1050
Total collected posts: 2
Next page: tid=39007861

Total shown posts: 1100
Total collected posts: 2
Next page: tid=39007811
URL: https://pantip.com/topic/39007782
Title: แฟนปีศาจแดงยืดอกภูมิใจ เด็กรุ่นใหม่ยังคงคิดเชียร์ Man.U
Date: 06/28/2019
Time: 17:42:38
Tags: สโมสรฟุตบอลอังกฤษ, ฟุตบอลต่างประเทศ, Premier, League, (League)
Post type: กระทู้ข่าว
Owner: Everyday I Die
Likes: 13
Emotes: 25
Story: แต่พอถามว่านักเตะคนโปรด คือใคร ?เค้ากลับตอบไม่ได้  อาจเพราะทั้งทีมสำคัญกว่านักเตะคนนึงก็เป็นได้


Total shown posts: 1150
Total collected posts: 3
Next page: tid=39007757

Total shown posts: 1200
Total collected


Total shown posts: 1550
Total collected posts: 7
Next page: tid=39007340

Total shown posts: 1600
Total collected posts: 7
Next page: tid=39007287

Total shown posts: 1650
Total collected posts: 7
Next page: tid=39007233

Total shown posts: 1700
Total collected posts: 7
Next page: tid=39007182

Total shown posts: 1750
Total collected posts: 7
Next page: tid=39007128

Total shown posts: 1800
Total collected posts: 7
Next page: tid=39007075

Total shown posts: 1850
Total collected posts: 7
Next page: tid=39007025

Total shown posts: 1900
Total collected posts: 7
Next page: tid=39006974

Total shown posts: 1950
Total collected posts: 7
Next page: tid=39006924

Total shown posts: 2000
Total collected posts: 7
Next page: tid=39006871

Total shown posts: 2050
Total collected posts: 7
Next page: tid=39006819

Total shown posts: 2100
Total collected posts: 7
Next page: tid=39006764
URL: https://pantip.com/topic/39006741
Title: “HOMESTAY”ได้ไปฉายที่ญี่ปุ่นแล้วว!!!
Date: 06/28/2019
Time: 11:03:


Total shown posts: 2500
Total collected posts: 14
Next page: tid=39006319

Total shown posts: 2550
Total collected posts: 14
Next page: tid=39006269

Total shown posts: 2600
Total collected posts: 14
Next page: tid=39006212
URL: https://pantip.com/topic/39006176
Title: 🍭[TWICE] 🍭 ในลุคสาวออฟฟิศทั้ง 9 สไตล์ ถ้าคัมแบ้คคอนเซปนี้จะเป็นยังไงนะ?
Date: 06/28/2019
Time: 04:50:38
Tags: TWICE, (นักร้องนักดนตรี), Girl, Group, K-POP
Post type: กระทู้สนทนา
Owner: momo_jinno
Likes: 17
Emotes: 36
Story: สาวๆรับสมัครวันซ์รุ่นที่ 3 ค่ะ
เล่นใหญ่มากกกกก ชอบสาวๆคนไหนในชุคสาวออฟฟิศบ้างคะ ถ้าทไวซ์มาคอนเซปนี้คงแหวกน่าดู


Total shown posts: 2650
Total collected posts: 15
Next page: tid=39006161
URL: https://pantip.com/topic/39006116
Title: ***** กระทู้โพสต์ไปเรื่อย ครั้งที่ ๑๗๙/๓๖๕ วัน ๓๖๕ ภาพ ปีที่ ๑๐ ประจำวันศุกร์ที่ ๒๘ มิถุนายน ๒๕๖๒ *****
Date: 06/28/2019
Time: 02:07:33
Tags: กล้องถ่ายรูป, กล้องดิจิตอล, การตกแต่งรูปภาพ, ทริปถ่ายรูป, ภาพถ่าย
Post type: กระทู้สนทนา
Owner: olerover
Likes: 11
Emotes: 13
Stor

Title: ยาขยันของทหารนาซีเยอรมันและทหารฝ่ายสัมพันธมิตร
Date: 06/27/2019
Time: 21:54:05
Tags: อาวุธยุทโธปกรณ์, ประวัติศาสตร์, วิทยาศาสตร์
Post type: กระทู้สนทนา
Owner: ravio
Likes: 15
Emotes: 20
Story: ทหารอเมริกันยิงปืนครก ใกล้กับเมือง Rhine, 1945
Credit: Thirteen Productions LLC




ในช่วงสงครามโลกครั้งที่  2
ความขยันอดทนอันน่าทึ่งของทหารนาซีเยอรมันและทหารสัมพันธมิตร
ต่างใช้ยาที่มีส่วนผสมลับ/ยาเสริมประสิทธิภาพ

ในช่วงทศวรรษที่ 1940 ทหารนาซีเยอรมัน
ได้รับยา Methamphetamine ชื่อ Pervitin
ในขณะที่ทหารอเมริกันและอังกฤษจะได้รับยา Amphetamine ชื่อ  Benzedrine

ทหารเสนารักษ์ทั้งสองฝ่ายต่างแจกจ่ายยาสารกระตุ้นเหล่านี้ - และอื่น ๆ เช่น Cocaine
เพื่อทำให้ทหารที่เหนื่อยล้าตื่นขึ้นมาอย่างกระฉับกระเฉงในวันที่ต้องสู้รบกันหนักมาก
เพื่อให้กองกำลังปฏิบัติการได้ยาวนานขึ้นภายใต้เงื่อนไขการลงโทษ/สู้แหลก
และเพื่อระงับ/บรรเทาความน่ากลัวและความอ่อนล้าต่าง ๆ
จากอาการบาดเจ็บและความเครียดหลังจากเกิดบาดแผล (PTSD)
ตามรายงานของ Secrets of the Dead: World War Speed
สารคดีเรื่องใหม่ที่ออกอากาศในวันที่ 25 มิถุนายน 201


Total shown posts: 3150
Total collected posts: 22
Next page: tid=39005640

Total shown posts: 3200
Total collected posts: 22
Next page: tid=39005590

Total shown posts: 3250
Total collected posts: 22
Next page: tid=39005538
URL: https://pantip.com/topic/39005527
Title: เปลี่ยนบ่อขยะเป็นบ่อปลา
Date: 06/27/2019
Time: 21:11:51
Tags: จัดสวน, เกษตรกรรม, ปลาสวยงาม, รีโนเวทบ้าน
Post type: กระทู้สนทนา
Owner: AOR_GANIC
Likes: 14
Emotes: 18
Story: หลังจากทำบ้านไก่ไปแล้ว คราวนี้มาทำบ่อปลากัน จากบ่อขยะน้ำดำปี๋ ยี้........
แรงบันดาลใจได้รับจากงู!  ใช่จ๊ะงู  งูเข้าบ้านเพราะข้างบ้านรกเหลือเกินไปดูพื้นที่กัน
ไงละ เหมาะเป็นเเหล่งชุมนุมงู ตะขาบ ตะเข็บ กิ้งกือ จริงๆ เดิมที่เมืออดีตกาลตรงนี้เป็นคลองแต่พามีถนนผ่านก็ปล่อยรกร้างอย่างที่เห็น ผักบุ้งหญ้าเต็ม เดินข้ามก็ไม่ได้จมโคลนอีก อันดับแรก หาดินหาทรายมาถมที่ซะก่อน
งานใช้แรงงานถามว่าถนัดมั้ย? ตอบเลยว่าไม่! แล้วทำไมไม่จ้าง งบหมด จบ! ทำเองต่อไป ชุดพร้อมลุยอาวุธมีจอบอันเดียวจริงๆ555ฮั่นเเน่...นั้นไงบ้านไก่ที่ทำในกระทู้ก่อน 
พื้นที่ช่างรกจริงๆ
จัดการกับผักบุ้ง


Total shown posts: 3600
Total collected posts: 24
Next page: tid=39005179

Total shown posts: 3650
Total collected posts: 24
Next page: tid=39005127

Total shown posts: 3700
Total collected posts: 24
Next page: tid=39005074

Total shown posts: 3750
Total collected posts: 24
Next page: tid=39005019

Total shown posts: 3800
Total collected posts: 24
Next page: tid=39004966

Total shown posts: 3850
Total collected posts: 24
Next page: tid=39004913
URL: https://pantip.com/topic/39004883
Title: [#RedVelvet] No.1 #Zimzalabim2ndwin🏆🏆on M! COUNTDOWN 190627
Date: 06/27/2019
Time: 17:46:48
Tags: K-POP, M, Countdown, (รายการโทรทัศน์), S.M., Entertainment, Red, Velvet, (นักร้องนักดนตรี), นักร้องนักดนตรีเกาหลี
Post type: กระทู้สนทนา
Owner: fangirlforGirlGroup
Likes: 14
Emotes: 35
Story: 


Total shown posts: 3900
Total collected posts: 25
Next page: tid=39004862

Total shown posts: 3950
Total collected posts: 25
Next page: tid=39004811
URL: https://pantip.com/topic/39004805
Title: ยอดวันนี้ หรั่งช

Title: EUN JIWON '불나방 (I'M ON FIRE) (Feat. Blue.D)' M/V
Date: 06/27/2019
Time: 16:02:43
Tags: K-POP, นักร้องนักดนตรีเกาหลี, YG, Entertainment, SECHSKIES, (นักร้องนักดนตรี)
Post type: กระทู้สนทนา
Owner: สมาชิกหมายเลข 4230246
Likes: 14
Emotes: 39
Story: EUN JIWON 'G1' SAMPLER

*เพลงworthless ฮันบินมีส่วนร่วมในส่วนของเนื้อเพลงและทำนองด้วยครับ

Spotify : https://open.spotify.com/album/7aW72lOTi7CPa04mDJDcI3?si=_IW33_jAT2KSwqFvjQGgwA

Itune :
https://music.apple.com/th/album/g1/1469501145?app=music&ign-mpt=uo%3D4


Total shown posts: 4150
Total collected posts: 31
Next page: tid=39004596
URL: https://pantip.com/topic/39004573
Title: [BR] บันเทิงใจไม่ไกลกรุง ละครป้องปาก "ตำนานมัจฉานุ" โรงมหรสพแห่งสยาม by GPX Legend150S
Date: 06/27/2019
Time: 15:54:00
Tags: มอเตอร์ไซค์, GPX, (มอเตอร์ไซค์), เที่ยวไทย
Post type: กระทู้รีวิว
Owner: สมัญตาชีวบุตร_omega_13
Likes: 16
Emotes: 18
Story: หาวเป็นดาวเดือนทินกร เขจรสว่างเวหา
เสร็จแล้วก็กลับลงมา ยังพื้นพสุธาทันที ฯ
บัดนั้น มัจฉานุผู้ชาญชัยศรี
เห็นประจักษ์

Title: แสดงความยินดีกับคู่บ่าวสาวด้วยครับ ขอบคุณพ่อและผู้ใหญ่ใจดีที่มอบโอกาส
Date: 06/27/2019
Time: 14:24:12
Tags: การเมือง
Post type: กระทู้คำถาม
Owner: น้อย NPR
Likes: 16
Emotes: 9
Story: พ่อนี้มีบุญคุณอันใหญ่หลวง
พ่อเฝ้าห้วงห่วงลูกแต่หลังเมื่อยังนอนเปล
พ่อเราเฝ้าโอ๋ละเห้
กล่อมลูกน้อยนอนเปลไม่ห่างหันเห ไปจนตาย


Total shown posts: 4400
Total collected posts: 35
Next page: tid=39004333

Total shown posts: 4450
Total collected posts: 35
Next page: tid=39004279
URL: https://pantip.com/topic/39004243
Title: @@ ทำไมกระทู้คำถามเกี่ยวกับ ส.ส.พรรคพลังประชารัฐและยาเสพติด @@ ถึงหายไปคะ????
Date: 06/27/2019
Time: 13:34:17
Tags: พรรคพลังประชารัฐ, พรรคอนาคตใหม่, ธนาธร, จึงรุ่งเรืองกิจ, สภาผู้แทน
Post type: กระทู้คำถาม
Owner: สมาชิกหมายเลข 1670436
Likes: 15
Emotes: 7
Story: ขอลบข้อความเพราะเว็บมาสเตอร์พันทิป

URL: https://pantip.com/topic/39004238
Title: ..........รักซ้อน ซ่อนพิษ........ตอนที่  ๒..........@@ โดย ลุงแผน
Date: 06/27/2019
Time: 13:29:30
Tags: แต่งนิยาย
Post type: กระทู้สนทนา
Owner: ล


Total shown posts: 4550
Total collected posts: 37
Next page: tid=39004178
URL: https://pantip.com/topic/39004151
Title: ตอนนี้ DTAC เอาอะไรไปสู้กับ TRUE , AIS ครับ ?
Date: 06/27/2019
Time: 12:56:07
Tags: dtac, truemove, H, AIS, e-Wallet, อินเทอร์เน็ต
Post type: กระทู้คำถาม
Owner: สมาชิกหมายเลข 5376307
Likes: 20
Emotes: 7
Story: มี Wallet ไหม
TRUE : มี True Money Wallet
AIS : มี Rabbbit Line Pay, MPAY
DTAC : ...

อยากดูทีวีอะทำไง
TRUE : มี True ID TV, True Vision
AIS : มี AIS Play Box
DTAC : ...

ทำ Digital Content ไหม
True : ทำลง True ID
AIS : ทำฉายผ่าน AIS Playbox
DTAC : ...

เน็ตบ้านแรงขนาดไหน
TRUE : True Online ความเร็วสูงสุด 1,000 Mbps
AIS : AIS Fiber ความเร็วสูงสุด 1,000 Mbps
DTAC : dtac@Home ความเร็วสูงสุด 15 Mbps

นี่ไม่พูดถึงสัญญาณอีกนะ

หรือทำแล้วแต่ผมไม่รู้ ต้องประชาสัมพันธ์นะ


Total shown posts: 4600
Total collected posts: 38
Next page: tid=39004128

Total shown posts: 4650
Total collected posts: 38
Next page: tid=39004070

Total shown posts: 4700
Total collected posts: 38